In [ ]:
from jcopdl.callback import set_config, Callback
import matplotlib.pyplot as plt

'1.1.1'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [46]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

bs = 128
crop_size = 224

train_transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(crop_size, scale=(0.8, 1.0)),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(235),
    transforms.CenterCrop(crop_size),
    transforms.ToTensor()])

trainset = datasets.ImageFolder('data/data/train/', transform=train_transform)
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=2)
testset = datasets.ImageFolder('data/data/test/', transform=test_transform)
testloader = DataLoader(testset, batch_size=bs, shuffle=False, num_workers=2)

In [47]:
label2cat, idx_class = trainset.classes, trainset.class_to_idx

In [48]:
feature, target = next(iter(trainloader))
feature.shape, target.shape

(torch.Size([128, 3, 224, 224]), torch.Size([128]))

# arsitektur dan config

In [49]:
def conv_block(in_feature, out_feature, padding=1, stride=1,
                  activation="relu", maxpool=True, kernel_size=3,
                  kernel_size_pool=2, pool_stride=2)-> list[nn.Sequential]:
    """
The conv_block function is designed to construct a sequential block of convolutional layers, complete with optional activation and max pooling. This is extremely useful for creating modular building blocks in Convolutional Neural Network (CNN) architectures, which we frequently utilize in notebooks and Kaggle.

Parameters
in_feature (int): The number of input channels (input features) for the convolutional layer.
out_feature (int): The number of output channels (output features) to be produced by the convolutional layer.
padding (int, optional, default=1): The amount of padding applied to the convolutional layer.
stride (int, optional, default=1): The stride for the convolutional kernel's movement.
activation (str, optional, default="relu"): The type of activation function to use after the convolutional layer. Currently supports "relu".
maxpool (bool, optional, default=True): If True, a max pooling layer will be appended after the activation.
kernel_size (int, optional, default=3): The kernel (filter) size for the convolutional layer.
kernel_size_pool (int, optional, default=2): The kernel size for the max pooling layer, if maxpool is enabled.
pool_stride (int, optional, default=2): The stride for the max pooling layer, if maxpool is enabled.
Returns
nn.Sequential: Returns a PyTorch nn.Sequential object containing the following sequence of layers:
A Convolutional layer (nn.Conv2d)
An Activation function (e.g., nn.ReLU), if specified
A Max Pooling layer (nn.MaxPool2d), if maxpool is True
    """
    layers = [nn.Conv2d(in_feature, out_feature, kernel_size=kernel_size, padding=padding, stride=stride)]
    if activation == "relu":
        layers.append(nn.ReLU())
    elif activation == "leakyrelu":
        layers.append(nn.LeakyReLU())
    elif activation == "sigmoid":
        layers.append(nn.Sigmoid())
    elif activation == "tanh":
        layers.append(nn.Tanh())
    if maxpool:
        layers.append(nn.MaxPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
    else:
        layers.append(nn.AvgPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
    return nn.Sequential(*layers)



def linear_block(in_features, out_features, activation='relu', dropout=0.0):
    layers = [nn.Linear(in_features, out_features)]
    # if batch_norm:
    #     layers.append(BatchNorm1d(out_features))
    if activation == 'relu':
        layers.append(nn.ReLU())
    elif activation == 'sigmoid':
        layers.append(nn.Sigmoid())
    elif activation == 'tanh':
        layers.append(nn.Tanh())
    elif activation == 'leakyrelu':
        layers.append(nn.LeakyReLU())
    elif activation == 'softmax':
        layers.append(nn.Softmax(dim=1))
    elif activation == 'elu':
        layers.append(nn.ELU())
    elif activation == 'selu':
        layers.append(nn.SELU())
    elif activation == 'lsoftmax':
        layers.append(nn.LogSoftmax(dim=1))
    if dropout > 0.0:
        layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)


In [50]:
class CNN(nn.Module):
    def __init__(self, dropout=0.0):
        super().__init__()
        self.conv = nn.Sequential(
            conv_block(3, 8),
            conv_block(8, 16),
            conv_block(16, 32),
            conv_block(32, 64),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            linear_block(64 * 14 * 14, 512, activation='relu', dropout=dropout),
            linear_block(512, 256, activation='relu', dropout=dropout),
            linear_block(256, len(label2cat), activation='lsoftmax')
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [51]:
config = set_config({
    "batch_size": bs,
    "crop_size": crop_size
    })

# Model

In [53]:
model = CNN().to(device)
critetion = nn.NLLLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
callback = Callback(model, config, outdir='models' )

# Training loop

In [54]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    from tqdm.auto import tqdm
    if mode == 'train':
        model.train()
    elif mode == 'test':
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)

        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(dim=1) == target).sum().item()
    cost = cost / len(dataset)
    accuracy = correct / len(dataset)
    return cost, accuracy

In [ ]:
while True:
    train_cost, train_acc = loop_fn('train', trainset, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_acc = loop_fn('test', testset, testloader, model, criterion, optimizer, device)

    callback.log(train_cost=train_cost, test_cost=test_cost, test_score=test_acc, train_score=train_acc)
    callback.save_checkpoint()
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()
    if callback.early_stopping(model, monitor='test_score', load_best_when_stop=True):
        print("Early stopping triggered.")
        callback.plot_cost()
        callback.plot_score()
        break





    print(f'Train Cost: {train_cost:.4f}, Test Cost: {test_cost:.4f}')